In [3]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results/execute_descentes.csv')


In [4]:
df.head()

,Instance,Algorithme,Seed,Score,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,best_improver_swap_descente,50,1723,4.420
1,instances/atsp_rand_50_75.txt,best_improver_swap_descente,51,1769,4.352
2,instances/atsp_rand_50_75.txt,best_improver_swap_descente,52,1768,4.562
3,instances/atsp_rand_50_75.txt,best_improver_swap_descente,53,1743,4.783
4,instances/atsp_rand_50_75.txt,best_improver_swap_descente,54,1744,4.816


In [5]:
neighbor_relations = ["swap","2opt"]
choice_mehod = ['first', 'best', 'worst']
versus = [('first', 'best'), ('first', 'worst'),('best', 'worst')]
instances = df['Instance'].unique()

In [6]:
def get_score(instance,neighbor_relation,choice_mehod,seed):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"{choice_mehod}_improver_{neighbor_relation}_descente") &
    (df['Seed'] == seed)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,choice_mehod,seed)
    

In [7]:
def gagne(instance,neighbor_relation,seed, vs):
    #print("vs:",vs)
    s1 = get_score(instance,neighbor_relation,vs[0],seed)
    s2 = get_score(instance,neighbor_relation,vs[1],seed)
    return s1 < s2

In [8]:
get_score("instances/atsp_rand_50_75.txt","2opt","first",55)

1940

In [9]:
gagne("instances/atsp_rand_50_75.txt","2opt",50, ("best","first"))

True

In [10]:
result_tuple={"instances" : instances}
for instance in instances:
    #print(instance)
    for nr in neighbor_relations:
        #print(nr)
        for vs in versus:
            #print(vs," : ",end="")
            r=[0,0]
            for seed in df["Seed"].unique():
                if gagne(instance,nr,seed, vs):
                    r[0]+=1
                else:
                    r[1]+=1
            try:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"].append((r[0],r[1]))
            except:
                result_tuple[f"{nr}_{vs[0]}_vs_{vs[1]}"] = [(r[0],r[1])]
            #print(r)


result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_first_vs_best,swap_first_vs_worst,swap_best_vs_worst,2opt_first_vs_best,2opt_first_vs_worst,2opt_best_vs_worst
0,instances/atsp_rand_50_75.txt,"(6, 4)","(5, 5)","(4, 6)","(7, 3)","(8, 2)","(6, 4)"
1,instances/atsp_rand_50_100.txt,"(5, 5)","(6, 4)","(5, 5)","(9, 1)","(10, 0)","(7, 3)"
2,instances/atsp_rand_60_50.txt,"(4, 6)","(3, 7)","(5, 5)","(8, 2)","(10, 0)","(7, 3)"
3,instances/atsp_rand_60_100.txt,"(4, 6)","(5, 5)","(5, 5)","(9, 1)","(10, 0)","(7, 3)"
4,instances/atsp_rand_70_55.txt,"(7, 3)","(4, 6)","(5, 5)","(10, 0)","(8, 2)","(3, 7)"
5,instances/atsp_rand_70_100.txt,"(5, 5)","(6, 4)","(4, 6)","(9, 1)","(10, 0)","(5, 5)"
6,instances/atsp_rand_80_70.txt,"(3, 7)","(5, 5)","(3, 7)","(10, 0)","(10, 0)","(5, 5)"
7,instances/atsp_rand_80_100.txt,"(6, 4)","(5, 5)","(5, 5)","(9, 1)","(10, 0)","(4, 6)"


Moyenne

In [11]:
df_group = df[["Algorithme","Score"]].groupby("Algorithme")
df_group.mean()

,Score
Algorithme,
best_improver_2opt_descente,1801.5000
best_improver_swap_descente,1206.7625
first_improver_2opt_descente,1596.2125
first_improver_swap_descente,1205.8250
worst_improver_2opt_descente,1809.1750
worst_improver_swap_descente,1207.7250


Pour swap gagnant: first

Pourt 2opt gagnant: first


### Durée swap normale

In [16]:
df_group = df[["Algorithme","CPU-Used-Time (ms)"]][(df["Algorithme"].str.contains("swap"))].groupby("Algorithme")
df_group.mean()

,CPU-Used-Time (ms)
Algorithme,
best_improver_swap_descente,23.713987
first_improver_swap_descente,7.646788
worst_improver_swap_descente,885.963613


### Durée swap avec évaluation incrémentale

In [18]:
df2 = pd.read_csv('../execution_results2/execute_descentes_incremental.csv')
df2_group = df2[["Algorithme","CPU-Used-Time (ms)"]][(df["Algorithme"].str.contains("swap"))].groupby("Algorithme")
df2_group.mean()

,CPU-Used-Time (ms)
Algorithme,
best_improver_swap_descente,4.497738
first_improver_swap_descente,0.354200
worst_improver_swap_descente,50.475100


Scores pratiquement inchangés

In [15]:
df2_group = df2[["Algorithme","Score"]].groupby("Algorithme")
df2_group.mean()

,Score
Algorithme,
best_improver_2opt_descente,1801.5000
best_improver_swap_descente,1162.6250
first_improver_2opt_descente,1596.2125
first_improver_swap_descente,1205.8250
worst_improver_2opt_descente,1809.1750
worst_improver_swap_descente,1207.7250
